[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ffserro/MVP/blob/master/mvp.ipynb)


# Regressão Linear para Series Temporais - Planejamento dos dispêndios de alimentação de militares da Marinha do Brasil

## Contextualização



<div align="justify" font-size="40px">
O planejamento eficiente dos recursos logísticos é um dos pilares para a manutenção da prontidão e da capacidade operacional das Forças Armadas. Entre os diversos insumos estratégicos, a alimentação das organizações militares desempenha papel central, tanto no aspecto orçamentário quanto no suporte direto às atividades diárias. Na Marinha do Brasil, a gestão dos estoques e dos gastos com gêneros alimentícios envolve múltiplos órgãos e abrange um volume expressivo de transações financeiras e contábeis, tornando-se um processo complexo e suscetível a variações sazonais, econômicas e administrativas.

Neste cenário, prever com maior precisão os custos relacionados ao consumo de alimentos é fundamental para otimizar a alocação de recursos públicos, reduzir desperdícios, evitar rupturas de estoque e aumentar a eficiência do planejamento orçamentário. Tradicionalmente, esse processo é conduzido por meio de análises históricas e técnicas de planejamento administrativo. No entanto, tais abordagens muitas vezes não capturam adequadamente os padrões temporais e as variáveis externas que influenciam os gastos.

A ciência de dados, e em particular as técnicas de modelagem de séries temporais, surge como uma alternativa poderosa para aprimorar esse processo decisório. Modelos como SARIMA, Prophet, XGBoost e LSTM permitem identificar tendências, sazonalidades e anomalias nos dados, possibilitando não apenas previsões mais robustas, mas também a geração de insights que subsidiam políticas de abastecimento e aquisição.

Assim, o presente trabalho propõe a aplicação de técnicas de análise e previsão de séries temporais sobre os dados históricos de consumo de alimentos da Marinha do Brasil, com o objetivo de estimar os custos futuros e explorar padrões relevantes que possam apoiar o processo de gestão logística e orçamentária. A relevância deste estudo reside não apenas no ganho potencial de eficiência administrativa, mas também na contribuição para a transparência, a racionalização do gasto público e a modernização da gestão de suprimentos em instituições estratégicas para o país.
</div>

## Glossário


* Municiamento
* Rancho
* Etapa
* Comensal
* Série Temporal
* Tendência
* Sazonalidade
* Estacionariedade


## Trabalho

In [203]:
#@title Download dos dados
!git clone 'https://github.com/ffserro/MVP.git'
!pip install -r '/content/MVP/requirements.txt' > '/content/pip_log.txt'
!if grep -iq "downloading" '/content/pip_log.txt';then python -c "import os;print('Por favor, reinicie a sessão e execute novamente.');os.kill(os.getpid(), 9)"; else python -c "print('Vamos começar!')"; fi

fatal: destination path 'MVP' already exists and is not an empty directory.
Vamos começar!


In [204]:
#@title Import de bibliotecas
from glob import glob

from datetime import date as dt, timedelta as td

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

from geopy.geocoders import Nominatim

import pmdarima as pm
from prophet import Prophet
from xgboost import XGBRegressor

from warnings import filterwarnings
filterwarnings('ignore')

In [209]:
mmm = pd.DataFrame()
mmm = pd.concat([mmm]+[pd.read_excel(d, parse_dates=[['ano', 'mes']]) for d in glob('/content/MVP/dados/mmm/*')]).reset_index(drop=True)

etapas = pd.DataFrame()
etapas = pd.concat([etapas]+[pd.read_excel(d, parse_dates=[['ano', 'mes']]) for d in glob('/content/MVP/dados/etapas/*')]).reset_index(drop=True).rename(columns={'uasg':'codigo'})

centralizadas = pd.read_csv('/content/MVP/dados/REL_OM_CENTRALIZADA_202509051127.csv', parse_dates=['DATA_INICIO', 'DATA_FIM'])
centralizadas = centralizadas[~(centralizadas.TIPO_CENTRALIZACAO_ID==4)]
centralizadas.drop(columns=['CONTATO', 'TELEFONE', 'CRIACAO', 'MODIFICACAO', 'TIPO_CENTRALIZACAO_ID', 'GESTORIA_ID'], inplace=True)
for codigo in centralizadas.OM_CENTRALIZADA_ID.unique():
  if centralizadas.OM_CENTRALIZADA_ID.value_counts().loc[codigo] > 1:
    centralizadas.loc[centralizadas[centralizadas.OM_CENTRALIZADA_ID==codigo].sort_values(by='DATA_INICIO').index, 'DATA_FIM'] = centralizadas[centralizadas.OM_CENTRALIZADA_ID==codigo].sort_values(by='DATA_INICIO').DATA_INICIO.shift(-1)
centralizadas.DATA_FIM.fillna(dt(2026, 1,1), inplace=True) # preenchendo os Not a Time com uma data no futuro. (É o caso das que não tem data fim)

om_info = pd.read_csv('/content/MVP/dados/OM_202509051130.csv')

In [226]:
om_info.head()

,DN_ID,AREA_ID,ID,CODIGO,NOME,TIPO,INDICATIVO_NAVAL,SIGLA,COMIMSUP,ODS,...,COD_SQ_LOCAL,TIPO_CONEXAO,CNPJ,ENDERECO,BAIRRO,CIDADE,UF,CEP,TELEFONE,SUB_DN_ID
0,4.0,2.0,10037,84330,CAPITANIA DOS PORTOS DO PIAUI,T,CPPARN,CPPI,NaN,84000.0,...,37.0,C,3.945020e+11,"AVENIDA DAS NAÇÕES UNIDAS, Nº 530",NOSSA SENHORA DO CARMO,PARNAIBA,PI,64200040,(86) 3321.2770,4.0
1,1.0,2.0,10039,91622,FRAGATA DODSWORTH,N,FDODSW,FDODSWORTH,NaN,NaN,...,39.0,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,7.0,2.0,10043,10000,COMANDANTE DA MARINHA,T,COMMAR,CM,NaN,NaN,...,43.0,C,NaN,ESPLANADA DOS MINISTERIOS - BLOCO N - 2o e 3o ...,NaN,BRASÍLIA,DF,70055900,NaN,7.0
3,1.0,2.0,10044,91901,CENTRO DE ADESTRAMENTO ALMIRANTE MARQUES DE LEÃO,I,CALEAO,CAAML,NaN,NaN,...,44.0,C,3.945020e+11,"ILHA DO MOCANGUÊ, S/Nº",NITERÓI,NITERÓI,RJ,24040300,(0 XX 21) 2716-1325 Retelma: 8116-1325,30.0
4,1.0,2.0,10045,91600,COMANDO DA FORÇA DE SUPERFÍCIE,T,FORPER,ComForSup,NaN,NaN,...,45.0,C,3.945020e+11,ILHA DE MOCANGUÊ,CENTRO,NITERÓI,RJ,24040300,NaN,1.0


In [259]:
import requests
url = 'https://maps.googleapis.com/maps/api/geocode/json'
params = {'sensor': 'false', 'address': 'Avenida Rio Branco 1, Rio de Janeiro'}
r = requests.get(url, params=params)

In [261]:
r.json()

{'error_message': 'You must use an API key to authenticate each request to Google Maps Platform APIs. For additional information, please refer to http://g.co/dev/maps-no-account',
 'results': [],
 'status': 'REQUEST_DENIED'}

In [256]:
om_info.ENDERECO

,ENDERECO
0,"AVENIDA DAS NAÇÕES UNIDAS, Nº 530"
1,NaN
2,ESPLANADA DOS MINISTERIOS - BLOCO N - 2o e 3o ...
3,"ILHA DO MOCANGUÊ, S/Nº"
4,ILHA DE MOCANGUÊ
...,...
546,"Ilha de Mocanguê Grande, nº 0"
547,Estrada Prefeito Wilson Pedro Francisco S/Nº
548,"Avenida Cidade de Santos, S/N°"
549,Ilha de Mocanguê Pequeno - S/N°


In [255]:
from geopy.geocoders import Nominatim
import time
from warnings import filterwarnings
filterwarnings('ignore')

from geo

geolocator = Nominatim(user_agent="cep_om", timeout=10)

locations = {}
for i, cep in enumerate(om_info.CEP.unique()):
  locations[cep] = geolocator.geocode(cep)
  print(i)
  time.sleep(1)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=64200040&format=json&limit=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7c6af98e4d10>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [249]:
(len(om_info.CEP))

551

In [247]:
locations = {}

for cep in om_info.CEP.unique():
  if cep not in locations:
    locations[cep] = Thread(target=get_loc, args=(cep,))
    locations[cep].start()

Exception in thread Thread-4 (get_loc):
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/geopy/geocoders/base.py", line 368, in _call_geocoder
    result = self.adapter.get_json(url, timeout=timeout, headers=req_headers)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/geopy/adapters.py", line 472, in get_json
    resp = self._request(url, timeout=timeout, headers=headers)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/geopy/adapters.py", line 500, in _request
    raise AdapterHTTPError(
geopy.adapters.AdapterHTTPError: Non-successful status code 403

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.12/dist-packages/ipykernel/ipkernel.py", 

In [208]:
import time

enderecos = []

for _, row in om_info.iterrows():
  enderecos.append(geolocator.geocode(row.ENDERECO))
  print(_)

0
1
2
3
4
5


6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70


71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103


104


105
106
107
108
109
110
111


...

159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190


191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306


307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423


424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=1%C2%B0+DN&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=5)"))

In [ ]:
enderecos

In [ ]:
centralizadas.TIPO_CENTRALIZACAO_ID.value_counts()

In [ ]:
# Dicionário para converter do código interno para número da uasg
tradutor_id_codigo = {row.ID: row.CODIGO for _, row in om_info.iterrows()}

centralizadas[['OM_CENTRALIZADA_ID', 'OM_CENTRALIZADORA_ID']] = centralizadas[['OM_CENTRALIZADA_ID', 'OM_CENTRALIZADORA_ID']].replace(tradutor_id_codigo)

In [ ]:
# Filtrando apenas as OM que possuem dados de todo o período observado (80 meses)
mmm = mmm[(mmm.codigo.isin((mmm.codigo.value_counts()==80).index)) & ~(mmm.nome.str.contains('TREINA'))]

In [ ]:
etapas[etapas.codigo.isin(mmm.codigo.unique())]

In [ ]:
centralizadas[centralizadas.OM_CENTRALIZADA_ID==14000]

In [ ]:
# etapas['centralizadoras'] = [centralizadas[(centralizadas.DATA_INICIO<=row.ano_mes) & (centralizadas.DATA_FIM>row.ano_mes) & (centralizadas.OM_CENTRALIZADA_ID==row.codigo)].OM_CENTRALIZADORA_ID.values for _, row in etapas.iterrows()]

merged = pd.merge(etapas, centralizadas, how='left',
                  left_on='codigo', right_on='OM_CENTRALIZADA_ID')

# Filtrando as linhas onde as datas atendem as condições (usando a máscara booleana)
merged = merged[(merged['DATA_INICIO'] <= merged['ano_mes']) &
                (merged['DATA_FIM'] > merged['ano_mes'])]

# Agora, podemos simplesmente atribuir a coluna 'OM_CENTRALIZADORA_ID'
etapas['centralizadoras'] = merged.groupby('codigo')['OM_CENTRALIZADORA_ID'].first().values

del merged


In [ ]:
etapas

In [ ]:
centralizadas[centralizadas.OM_CENTRALIZADA_ID==11100]

In [ ]:
tradutor_id_codigo

In [ ]:
centralizadas

In [ ]:
etapas

In [ ]:
mmm_custo_unitario = pd.merge(left=mmm, right=etapas.groupby(['ano', 'mes', 'codigo']).quantidade.sum().reset_index(), how='inner', on=['codigo', 'ano', 'mes'])[['ano', 'mes', 'codigo', 'nome', 'totais_balanco_paiol_despesa', 'quantidade']]

In [ ]:
mmm_custo_unitario['custo_unitario'] = mmm_custo_unitario.totais_balanco_paiol_despesa / mmm_custo_unitario.quantidade

In [ ]:
mmm_custo_unitario.sort_values(by='custo_unitario').tail(50)

In [ ]:
mmm_custo_unitario.custo_unitario.describe()

In [ ]:
etapas[etapas.codigo_etapa < 600].groupby(['ano', 'mes', 'codigo']).quantidade.sum().reset_index()

In [ ]:
mmm_marinha = mmm.groupby(['ano', 'mes'])[[col for col in mmm.columns if col not in ['ano', 'mes', 'codigo', 'nome']]].sum().reset_index()

In [ ]:
mmm_marinha['mes'] = [''.join([str(i[0]), '_', str(i[1])]) for i in zip(mmm_marinha.mes.values, mmm_marinha.ano.values)]
mmm_marinha.drop(columns=['ano'], inplace=True)
mmm_marinha = mmm_marinha.iloc[:-2]

In [ ]:
mmm_etapas = pd.merge(left=mmm, right=etapas, how='inner', on=['ano', 'mes', 'codigo'])

In [ ]:
mmm_etapas[mmm_etapas.codigo_etapa.isin([103, 105])][['ano', 'mes', 'nome', 'codigo_etapa', 'quantidade']]

In [ ]:
pd.DataFrame({
    'diferenca': mmm_receita_despesa.despesa_autorizada_global - (mmm_receita_despesa.generos_consumidos + mmm_receita_despesa.vale_extra),
    'sobra_licita': mmm_receita_despesa.sobra_licita
})

In [ ]:
da_marinha = da.groupby(['mes', 'ano']).despesa_autorizada_global.sum().reset_index()
da_marinha['mes'] = ['_'.join(['{:02d}'.format(i[0]), str(i[1])]) for i in zip(da_marinha.mes.values, da_marinha.ano.values)]
da_marinha = da_marinha.sort_values(by=['ano', 'mes']).iloc[:-2]
fig = px.line(
    da_marinha,
    x = 'mes',
    y = 'despesa_autorizada_global',
    title='Despesa Autorizada global'
)

fig.show()

In [ ]:
def grafico_base(titulo):
    return px.line(
        mmm_marinha,
        x = 'mes',
        y = 'totais_balanco_paiol_despesa',
        labels = {
            'mes': 'Mês e ano',
            'totais_balanco_paiol_despesa': 'Totais das despesas'
        },
        title = titulo
    )

fig = grafico_base('Gastos com alimentação dos últimos cinco anos')

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
# Teste de estacionariedade

result = adfuller(mmm_marinha.totais_balanco_paiol_despesa)
print(f'ADF: {result[0]}, p-valor: {result[1]}')

In [ ]:
naive_forecast = mmm_marinha.totais_balanco_paiol_despesa.shift(1)
mae_naive = mean_absolute_error(mmm_marinha.totais_balanco_paiol_despesa.iloc[1:], naive_forecast.iloc[1:])
print('Baseline Naïve MAE:', mae_naive)

In [ ]:
# SARIMA

train = mmm_marinha.iloc[:-12, :]
test = mmm_marinha.iloc[-12:, :]

model_auto = pm.auto_arima(train.totais_balanco_paiol_despesa, seasonal=True, m=12, stepwise=True, trace=True)
print(model_auto.summary())

In [ ]:
mmm

In [ ]:
sarima_model = SARIMAX(train.totais_balanco_paiol_despesa,
                       order=model_auto.order,
                       seasonal_order=model_auto.seasonal_order)

sarima_fit = sarima_model.fit(disp=False)

forecast_sarima = sarima_fit.get_forecast(steps=12)
pred_sarima = forecast_sarima.predicted_mean

mae_sarima = mean_absolute_error(test.totais_balanco_paiol_despesa, pred_sarima)
print('SARIMA MAE:', mae_sarima)



In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo SARIMA')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = pred_sarima
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
# Previsão com o Prophet

prophet_df = mmm_marinha[['mes', 'totais_balanco_paiol_despesa']]
prophet_df.columns = ['ds', 'y']
prophet_df['ds'] = pd.to_datetime(prophet_df['ds'], format='%m_%Y')

model_prophet = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
model_prophet.fit(prophet_df)

future = model_prophet.make_future_dataframe(periods=12, freq='M')
forecast = model_prophet.predict(future)

forecast_test = forecast.set_index('ds').loc[pd.to_datetime(test.mes, format='%m_%Y')]
mae_prophet = mean_absolute_error(test['totais_balanco_paiol_despesa'], forecast_test['yhat'])
print('Prophet MAE', mae_prophet)

In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo Prophet')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = forecast_test.yhat
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
# Regressão com XGBoost para séries temporais

xg_df = mmm_marinha[['mes', 'totais_balanco_paiol_despesa']]
xg_df['ano'] = xg_df['mes'].apply(lambda x: x.split('_')[-1]).astype(int)
xg_df['mes'] = xg_df['mes'].apply(lambda x: x.split('_')[0]).astype(int)
xg_df['lag1'] = xg_df['totais_balanco_paiol_despesa'].shift(1)
xg_df['lag3'] = xg_df['totais_balanco_paiol_despesa'].shift(3)
xg_df['lag6'] = xg_df['totais_balanco_paiol_despesa'].shift(6)
xg_df['rolling3'] = xg_df['totais_balanco_paiol_despesa'].rolling(3).mean()
xg_df['rolling6'] = xg_df['totais_balanco_paiol_despesa'].rolling(6).mean()

xg_df = xg_df.dropna().reset_index(drop=True)

In [ ]:
train_xg = xg_df.iloc[:-12]
test_xg = xg_df.iloc[-12:]

X_train = train_xg.drop(columns=['totais_balanco_paiol_despesa'])
y_train = train_xg['totais_balanco_paiol_despesa']
X_test = test_xg.drop(columns=['totais_balanco_paiol_despesa'])
y_test = test_xg['totais_balanco_paiol_despesa']

xgb = XGBRegressor(
    n_estimators = 300,
    learning_rate = 0.05,
    max_depth = 5,
    subsample = 0.8,
    colsample_bytree = 0.8,
    random_state = 42
)

xgb.fit(X_train, y_train)

pred_xgb = xgb.predict(X_test)

mae_xgb = mean_absolute_error(y_test, pred_xgb)

print("XGBoost MAE:", mae_xgb)


In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo XGBoost')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = pred_xgb
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
hw_model = ExponentialSmoothing(
    train['totais_balanco_paiol_despesa'],
    trend='add',
    seasonal='add',
    seasonal_periods=12
).fit()

pred_hw = hw_model.forecast(12)

In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo ExponentialSmoothing')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = pred_hw
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
despesas_scaled = scaler.fit_transform(mmm_marinha.totais_balanco_paiol_despesa.values.reshape(-1, 1))

def create_sequences(data, window=12):
    X, y = [], []
    for i in range(len(data)- window):
        X.append(data[i:i+window])
        y.append(data[i+window])
    return np.array(X), np.array(y)

X, y = create_sequences(despesas_scaled)


split = len(X) - 12
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
print('Shape treino:', X_train.shape, y_train.shape)

In [ ]:
model = Sequential()
model.add(LSTM(64, activation='tanh', return_sequences=True, input_shape=(12, 1)))
model.add(Dropout(0.2))
model.add(LSTM(32, activation='tanh'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mae')

history = model.fit(
    X_train, y_train,
    epochs = 200,
    batch_size = 4,
    validation_split = 0.1,
    verbose = 1
)

In [ ]:
y_pred = model.predict(X_test)

y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_inv = scaler.inverse_transform(y_pred)

mae_lstm = mean_absolute_error(y_test_inv, y_pred_inv)
print('LSTM MAE:', mae_lstm)

In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo LSTM')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = y_pred_inv.reshape(1, -1)[0]
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
def create_sequences_multistep(data, window=12, horizon=12):
    X, y = [], []
    for i in range(len(data) - window - horizon + 1):
        X.append(data[i:i+window])
        y.append(data[i+window:i+window+horizon].flatten())
    return np.array(X), np.array(y)

window = 12
horizon = 12
X, y = create_sequences_multistep(despesas_scaled, window, horizon)

split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
print('X_train shape:', X_train.shape)
print('y_train, shape:', y_train.shape)

In [ ]:
model = Sequential()
model.add(LSTM(64, activation="tanh", return_sequences=True, input_shape=(window, 1)))
model.add(Dropout(0.2))
model.add(LSTM(32, activation="tanh"))
model.add(Dense(horizon))  # saída com 12 valores (multi-step)

model.compile(optimizer="adam", loss="mse")

history = model.fit(
    X_train, y_train,
    epochs=300,
    batch_size=4,
    validation_split=0.1,
    verbose=1
)

In [ ]:
y_pred = model.predict(X_test)

y_test_inv = scaler.inverse_transform(y_test)
y_pred_inv = scaler.inverse_transform(y_pred)

mae_lstm_multi = mean_absolute_error(y_test_inv.flatten(), y_pred_inv.flatten())
print('LSTM Multi-step MAE:', mae_lstm_multi)

In [ ]:
last_input_dates = pd.to_datetime(mmm_marinha.mes, format='%m_%Y').iloc[-(window + horizon):-horizon]
future_dates = pd.date_range(start = pd.to_datetime(mmm_marinha.mes, format='%m_%Y').iloc[-horizon], periods=horizon, freq='M')

In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo LSTM Multi-step')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = y_pred_inv.flatten()
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
teste = ['A', 'B', 'C']

In [ ]:
model = 'LSTM' #@param ["SARIMAX", "prophet", "XGBoost", "ExponentialSmothing", "LSTM"]
test = 'A' #@param

print(f"Selected model: {modelo}")
print(f'Selected test : {test}')

In [ ]:
mmm['mes_ano'] = mmm.mes.astype(str) + '_' + mmm.ano.astype(str)

In [265]:
!playwright install chromium

173.7 MiB [] 0% 132.2s173.7 MiB [] 0% 53.5s173.7 MiB [] 0% 37.1s173.7 MiB [] 0% 23.8s173.7 MiB [] 0% 13.6s173.7 MiB [] 0% 10.4s173.7 MiB [] 1% 8.6s173.7 MiB [] 1% 7.8s173.7 MiB [] 1% 7.1s173.7 MiB [] 2% 6.8s173.7 MiB [] 2% 6.1s173.7 MiB [] 3% 6.2s173.7 MiB [] 3% 6.4s173.7 MiB [] 3% 6.6s173.7 MiB [] 4% 6.9s173.7 MiB [] 4% 7.1s173.7 MiB [] 4% 7.3s173.7 MiB [] 4% 7.5s173.7 MiB [] 4% 7.6s173.7 MiB [] 4% 7.8s173.7 MiB [] 4% 8.0s173.7 MiB [] 4% 8.3s173.7 MiB [] 5% 8.3s173.7 MiB [] 5% 8.2s173.7 MiB [] 5% 8.3s173.7 MiB [] 6% 8.3s173.7 MiB [] 6% 8.2s173.7 MiB [] 6% 8.1s173.7 MiB [] 6% 8.0s173.7 MiB [] 7% 7.7s173.7 MiB [] 7% 7.5s173.7 MiB [] 8% 7.2s173.7 MiB [] 8% 7.4s173.7 MiB [] 8% 7.2s173.7 MiB [] 8% 7.3s173.7 MiB [] 9% 7.1s173.7 MiB [] 10% 6.8s173.7 MiB [] 10% 6.6s173.7 MiB [] 11% 6.3s173.7 MiB [] 11% 6.0s173.7 MiB [] 12% 5.8s173.7 MiB [] 13% 5.7s173.7 MiB [] 13% 7.7s173.7 MiB [] 14% 7.6s173.7 MiB [] 14% 7.4s173.7 MiB [] 15% 7.3s173.7 MiB [] 15% 7.4s173.7 MiB [] 15% 7.3s173.7 MiB [] 15% 7.4s

In [268]:
from playwright.async_api import async_playwright
import time

In [269]:
with async_playwright() as p:
    browser = p.chromium.launch(headless=False)
    page = browser.new_page()

    page.goto('https://www.maps.google.com')

    page.locator('xpath=/html/body/div[1]/div[3]/div[9]/div[3]/div[1]/div[1]/div/div[2]/form/input').fill('Avenida Rio Branco 1, Rio de Janeiro')
    page.keyboard.press('Enter')
    time.sleep(1)
    print(page.url())

TypeError: 'PlaywrightContextManager' object does not support the context manager protocol

In [286]:
from playwright.async_api import async_playwright
import asyncio

async def get_latlong(endereco):
  async with async_playwright() as p:
    browser = await p.chromium.launch()
    page = await browser.new_page()
    await page.goto('https://www.google.com/maps')

    await page.locator('#searchboxinput').fill('Avenida Rio Branco 1, Rio de Janeiro')
    await page.keyboard.press('Enter')
    await page.wait_for_timeout(5000)
    time.sleep(1)
    return page.url.split('@')[1].split(',')[:2]

In [289]:
enderecos = {}

for endereco in om_info.ENDERECO.unique():
  enderecos[endereco] = await get_latlong(endereco)

IndexError: list index out of range

In [ ]:
enderecos